Collects data for file_properties_df.

file_properties_df holds metadata about data in our dataset,
details of metadata can be seen at src/nna/tests/mock_data.py::mock_file_properties_df


In [1]:
from pathlib import Path
import pandas as pd
import datetime

from collections import Counter

from nna.fileUtils import list_files, concurrent_get_media_duration, read_file_properties_v2


Output(layout=Layout(border='1px solid black'))

In [10]:
# External Programs
# ffprobe version >= 4.3.1
ffprobe_path = '/home/enis/sbin/ffprobe'

In [11]:
# PARAMETERS for new DATABASE

# increase version number accordinly
previous_database_ver_str = 'V102'
new_database_ver_str = 'Vtodd'
# where to save txt file storing length info
# old_data_folder="/home/enis/projects/nna/data/"
data_folder = '/scratch/enis/data/nna/database/'
#/scratch/enis/data/nna/database

# path to search for audio files
# where
# ignore_folders=['/tank/data/nna/real/stinchcomb/']

# search_path = "/tank/data/nna/real/"
search_path = '/tank/data/nna/recover_files/2022/'
ignore_folders = [
    "/tank/data/nna/real/stinchcomb/dups/",
    "/tank/data/nna/real/stinchcomb/excerpts/",
    "/tank/data/nna/real/stinchcomb/"
]
# search_path="/tank/data/nna/real/stinchcomb/"


In [12]:
# # PARAMETERS for external or small inferences

# # PARAMETERS

# # increase version number accordinly
# previous_database_ver_str = ''
# new_database_ver_str = 'V1'

# # where to save txt file storing length info
# # old_data_folder="/home/enis/projects/nna/data/"
# # data_folder = '/scratch/enis/data/nna/database/'
# data_folder = '/scratch/enis/data/nna/collar_database/'

# #/scratch/enis/data/nna/database

# # path to search for audio files
# # where
# # ignore_folders=['/tank/data/nna/real/stinchcomb/']

# # search_path="/tank/data/nna/real/"
# search_path = '/tank/data/nna/audio_collars/'

# ignore_folders=["/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]
# # search_path="/tank/data/nna/real/stinchcomb/"


In [13]:
# create Relative Path names

# if we already have a list of files we can load them
# files_list_path=data_folder+"stinchcomb_files_pathV1.txt"
files_list_path = data_folder + f"allFields_path{new_database_ver_str}.txt"

# if we calculated audio lengths and saved them into text file,
# we can load them
fileswlen_path = data_folder + f"allFields_wlen_f{new_database_ver_str}.txt"
filesWError_out = data_folder + f"allFields_wERROR_f{new_database_ver_str}.txt"

# do NOT add pkl extension at the end
pkl_file_name = f"allFields_data{new_database_ver_str}"

# this is the current info we have so we can check if we already processed a file before
current_pkl_file = data_folder + f"allFields_data{previous_database_ver_str}.pkl"
print(current_pkl_file)

/scratch/enis/data/nna/database/allFields_dataV102.pkl


In [14]:
# Find files
# in given search path ignoring given directories
if Path(fileswlen_path).exists():
    with open(files_list_path, "r") as f:
        lines = f.readlines()
        files_path_list = [line.strip() for line in lines]
else:
    files_path_list = list_files(search_path, ignore_folders)


In [15]:
print('example file:', files_path_list[0])
print('total number of files:', len(files_path_list))
assert Path(files_path_list[0]).is_dir() == False


example file: /tank/data/nna/recover_files/2022/folders_and_files_tree.txt
total number of files: 16628


In [16]:
# count file extensions
suffix2files = {}

files_suffixes = []
for m in files_path_list:
    m = Path(m)
    mSuffix = m.suffix.lower()
    files_suffixes.append(mSuffix)
    suffix2files.setdefault(mSuffix, []).append(m)

print(Counter(files_suffixes))


Counter({'.wav': 16523, '.sm4dump': 79, '.txt': 25, '.csv': 1})


In [17]:
#V4: Counter({'.flac': 112053, '.aac': 9101, '.mp3': 388, '.flac~': 1})
#V5: Counter({'.flac': 151527, '.aac': 9101, '.mp3': 388, '.flac~': 1})
#V6: Counter({'.flac': 165976, '.aac': 9101, '.mp3': 388, '.flac~': 1, '.txt': 1})
#V7: Counter({'.flac': 204692, '.aac': 9101, '.wav': 2340, '.mp3': 388, '.txt': 3, '.flac~': 1, '.filepart': 1})

#V8: Counter({'.flac': 227227, '.aac': 9101, '.mp3': 388, '.flac~': 1, '.filepart': 1, '.txt': 1})

#V10: Counter({'.flac': 259798})


In [18]:
#

if Path(current_pkl_file).exists():

    current_file_properties_df = pd.read_pickle(str(current_pkl_file))

    # remove files we already know about
    currentFileSet = set(current_file_properties_df.index)
    currentFileSet = set([str(m) for m in currentFileSet])
    foundFileSet = set(files_path_list)
    foundFileSet = foundFileSet.difference(currentFileSet)
    New_files_path_list = list(foundFileSet)
    a_str = "new", len(New_files_path_list), "previously", len(
        currentFileSet), "total", len(files_path_list)
    print(a_str)
else:
    print('does NOT exists', current_pkl_file)
    only_wav = [m for m in files_path_list if Path(m).suffix == '.wav']
    New_files_path_list = only_wav[:]


does NOT exists /scratch/enis/data/nna/database/allFields_dataV102.pkl


In [20]:
len(New_files_path_list)

16523

In [21]:
# usage
length_dict, filesWError = concurrent_get_media_duration(New_files_path_list)
# print('Files with duration:', length_dict)
# print('Files with errors:', filesWError)

Progress: 0.00%
Progress: 6.05%
Progress: 12.10%
Progress: 18.16%
Progress: 24.21%
Progress: 30.26%
Progress: 36.31%
Progress: 42.37%
Progress: 48.42%
Progress: 54.47%
Progress: 60.52%
Progress: 66.57%
Progress: 72.63%
Progress: 78.68%
Progress: 84.73%
Progress: 90.78%
Progress: 96.83%


In [29]:
# filesWError
# #
# for i in filesWError:
#     print(i[0])

In [25]:
filesWError[0]

('/tank/data/nna/recover_files/2022/region/11/2022/S4A10276_20220801_130002.wav',
 "Command '['/scratch/enis/conda/envs/speechEnv/bin/ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_format', '/tank/data/nna/recover_files/2022/region/11/2022/S4A10276_20220801_130002.wav']' returned non-zero exit status 1.")

In [26]:
# print and save files with errors
print(len(filesWError))
with open(filesWError_out, "w") as f:
    lines = [line[0] for line in filesWError]
    f.write("\n".join(lines) + "\n")
print('saved', filesWError_out)

114
saved /scratch/enis/data/nna/database/allFields_wERROR_fVtodd.txt


In [23]:
filesWError[0]

('/tank/data/nna/recover_files/2022/region/11/2022/S4A10276_20220801_130002.wav',
 "Command '['/scratch/enis/conda/envs/speechEnv/bin/ffprobe', '-v', 'quiet', '-print_format', 'json', '-show_format', '/tank/data/nna/recover_files/2022/region/11/2022/S4A10276_20220801_130002.wav']' returned non-zero exit status 1.")

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
test = length_dict.copy()

new_test = {}
for key, value in test.items():
    key = Path(key)
    key = (key.parts[:5] + key.parts[6:])
    key = Path(*key)
    new_test[key] = value
    # print(len(key.parents))
    # print(key.parts)
    # print(key.parts[6])
    # print(key.parts[5])
    # break
# new_test = {Path(*Path(key).parts[2:]): value for key, value in test.items()}


In [54]:
list(new_test.keys())[0]

PosixPath('data/nna/recover_files/2022/region/11/2022/S4A10276_20220815_130002.wav')

In [107]:
# file_properties
file_properties, exceptions = read_file_properties_v2(list(new_test.keys()),
                                                      debug=0)


In [45]:
# set([tuple(str(ex).split("/")[5:7]) for ex in exceptions])
# Splitting the path into its components


set()

In [108]:
len(exceptions), len(New_files_path_list)


(0, 16523)

In [110]:
for f, lengthSeconds in new_test.items():
    if file_properties.get(Path(f)) is not None:
        file_properties[Path(f)]["durationSec"] = lengthSeconds
        file_properties[Path(f)]["timestampEnd"] = file_properties[Path(
            f)]["timestamp"] + datetime.timedelta(seconds=lengthSeconds)
    else:
        print("file not found", f)
file_properties_df = pd.DataFrame(file_properties).T


In [111]:
file_properties_df['locationId'] = file_properties_df['region']
file_properties_df['region'] = 'region'


In [112]:
file_properties_df

,site_id,locationId,site_name,recorderId,hour_min_sec,year,month,day,region,timestamp,durationSec,timestampEnd
/tank/data/nna/recover_files/region/11/2022/S4A10276_20220815_130002.wav,11,region,,S4A10276,130002,2022,08,15,region,2022-08-15 13:00:02,3597.995828,2022-08-15 13:59:59.995828
/tank/data/nna/recover_files/region/11/2022/S4A10276_20220714_140002.wav,11,region,,S4A10276,140002,2022,07,14,region,2022-07-14 14:00:02,3597.995828,2022-07-14 14:59:59.995828
/tank/data/nna/recover_files/region/11/2022/S4A10276_20220731_090002.wav,11,region,,S4A10276,090002,2022,07,31,region,2022-07-31 09:00:02,3597.995828,2022-07-31 09:59:59.995828
/tank/data/nna/recover_files/region/11/2022/S4A10276_20220815_120000.wav,11,region,,S4A10276,120000,2022,08,15,region,2022-08-15 12:00:00,3599.998549,2022-08-15 12:59:59.998549
/tank/data/nna/recover_files/region/11/2022/S4A10276_20220717_030002.wav,11,region,,S4A10276,030002,2022,07,17,region,2022-07-17 03:00:02,3597.995828,2022-07-17 03:59:59.995828
...,...,...,...,...,...,...,...,...,...,...,...,...
/tank/data/nna/recover_files/region/35/2022/S4A10272_20220830_200002.wav,35,region,,S4A10272,200002,2022,08,30,region,2022-08-30 20:00:02,3597.995828,2022-08-30 20:59:59.995828
/tank/data/nna/recover_files/region/35/2022/S4A10272_20220829_110004.wav,35,region,,S4A10272,110004,2022,08,29,region,2022-08-29 11:00:04,3595.998912,2022-08-29 11:59:59.998912
/tank/data/nna/recover_files/region/35/2022/S4A10272_20220825_090002.wav,35,region,,S4A10272,090002,2022,08,25,region,2022-08-25 09:00:02,3597.995828,2022-08-25 09:59:59.995828
/tank/data/nna/recover_files/region/35/2022/S4A10272_20220831_020002.wav,35,region,,S4A10272,020002,2022,08,31,region,2022-08-31 02:00:02,3597.995828,2022-08-31 02:59:59.995828


In [49]:
assert (file_properties_df[file_properties_df['durationSec'] > 0]
       ).shape == file_properties_df.shape
## ignore files with 0 length
# file_properties_df = file_properties_df[file_properties_df['durationSec']>0]

In [61]:
## these files are too short, I should investiage these
# (file_properties_df[file_properties_df['durationSec']<21])

In [63]:
# merge with previous file properties
merged_file_properties_df = pd.concat(
    [file_properties_df, current_file_properties_df])


In [65]:
merged_file_properties_df.to_pickle(data_folder + pkl_file_name + ".pkl")


In [80]:
merged_file_properties_df.to_pickle(data_folder + pkl_file_name + "2.pkl")


## check dataset stats


In [81]:
pd.__version__

'1.2.0'

In [66]:
current_file_properties_df = pd.read_pickle(data_folder + pkl_file_name +
                                            ".pkl")


In [70]:
print('total days of recording',
      ((sum(current_file_properties_df['durationSec']) / 3600) / 24))


total days of recording 3536.5799098982093


In [71]:
import numpy as np

files_df_2019 = current_file_properties_df[current_file_properties_df['year'] ==
                                           '2019']
site_region = list({
    (val[1], val[0]) for val in (files_df_2019[['locationId', 'region']].values)
})
site_region.sort()

In [ ]:
for region, site_id in site_region:
    site_files_df_2019 = files_df_2019[files_df_2019['site_id'] == site_id]
    region_files_df_2019 = site_files_df_2019[site_files_df_2019['region'] ==
                                              region]
    t = region_files_df_2019['timestamp'].sort_values()[0]
    print(site_id, region, ':', t.month, t.day, '(M/D)')
    # print(region_files_df_2019['timestamp'].sort_values()[0])

In [52]:
file_properties_df[['site_id', 'locationId', 'timestamp', 'durationSec']]


,site_id,locationId,timestamp,durationSec
data/nna/recover_files/2022/region/11/2022/S4A10276_20220815_130002.wav,2022,2022,2022-08-15 13:00:02,3597.995828
data/nna/recover_files/2022/region/11/2022/S4A10276_20220714_140002.wav,2022,2022,2022-07-14 14:00:02,3597.995828
data/nna/recover_files/2022/region/11/2022/S4A10276_20220731_090002.wav,2022,2022,2022-07-31 09:00:02,3597.995828
data/nna/recover_files/2022/region/11/2022/S4A10276_20220815_120000.wav,2022,2022,2022-08-15 12:00:00,3599.998549
data/nna/recover_files/2022/region/11/2022/S4A10276_20220717_030002.wav,2022,2022,2022-07-17 03:00:02,3597.995828
...,...,...,...,...
data/nna/recover_files/2022/region/35/2022/S4A10272_20220830_200002.wav,2022,2022,2022-08-30 20:00:02,3597.995828
data/nna/recover_files/2022/region/35/2022/S4A10272_20220829_110004.wav,2022,2022,2022-08-29 11:00:04,3595.998912
data/nna/recover_files/2022/region/35/2022/S4A10272_20220825_090002.wav,2022,2022,2022-08-25 09:00:02,3597.995828
data/nna/recover_files/2022/region/35/2022/S4A10272_20220831_020002.wav,2022,2022,2022-08-31 02:00:02,3597.995828


In [121]:
import pandas as pd

# Convert the duration from seconds to hours
file_properties_df['durationHours'] = file_properties_df['durationSec'] / 3600

# Group by site_id and locationId, then sum the durations
result = file_properties_df.groupby([
    'site_id',
])['durationHours'].sum().reset_index()
result.sort_values(by='site_id', ascending=True, inplace=True)
result.reset_index(drop=True, inplace=True)

# print(result)

result.to_csv('/tank/data/nna/recover_files/2022/duration.csv', index=False)


In [120]:
df = file_properties_df.copy()
# Convert timestamp column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Extract year and month
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month

# Group by site_id, locationId, year and month, then sum the durations
result = df.groupby(['site_id', 'month'])['durationHours'].sum().reset_index()

result.sort_values(by='site_id', ascending=True, inplace=True)
result.reset_index(drop=True, inplace=True)

# print(result)

result.to_csv('/tank/data/nna/recover_files/2022/durations_bymonth.csv',
              index=False)
print(result)


   site_id  month  durationHours
0       11      7     323.808829
1       11      8     360.786849
2       11      9      34.830630
3       12      7     323.809384
4       12      8     336.804094
..     ...    ...            ...
62      34      8     367.783506
63      34      9       2.999442
64      35      8     305.820790
65      35      7     311.816896
66      35      9       4.997773

[67 rows x 3 columns]


In [119]:
result['year'].unique()

array([2022])